# MMOB Project

# import dataset

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [101]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
data_file ='https://raw.githubusercontent.com/DunodMax/London-Transportation-modeling/main/lpmc.dat' #import path 
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}

# function definition

In [102]:
def save_results(results,model_number):
    """This function is giving results and estimations (in a dict format) of a model, and is saving locally those results
    in a designated file with dynamic name (in a csv format)"""
    df = pd.DataFrame(results, index=[0])
    df.to_csv('estimation_model' + str(model_number) + '.csv', index=False, header=True)

In [103]:
def estimate_results(model,name):
    """Function that will estimate parameters of models. It will take an unsestimated biogeme model, will perform
    the estimation, and output the estimation on a table"""
    results = model.estimate()
    results.getEstimatedParameters()
    model.modelName=name
    all_results[name] = biogeme_model0.estimate()
    save_results(all_results,0)
    print(res.compileEstimationResults(all_results))
    return (results)
    

In [104]:
def ratio_test(best_results,tested_results,lvl_of_confidence):
    """This function will perform a Loglikelihood ration test with a 95% level of confidence and plot the conclusion"""
    test=tested_results.likelihood_ratio_test(best_results, lvl_of_confidence)
    return test
    

In [105]:
def download_results():
    """This function takes no argument but download the dictionnary called 'all_results' containing estimation
    and statistics as a csv, on a designated path"""
    df_results=pd.DataFrame.from_dict(all_results.items(), orient='columns')
    df_results.to_csv("results.csv")

# Model 0

all useful columns for this model are used as variables

In [106]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [107]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [108]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions definition

In [109]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

av is used to specify which alternative is available, where we considered any one can uses any transportation mode
and thus av takes a very simple format

In [110]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [111]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
results_0=estimate_results(biogeme_model0,'Model0')
print(all_results)

[15:01:45] < Warning >   You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]
                                       Model0
Number of estimated parameters       5.000000
Sample size                      81086.000000
Final log likelihood            -74975.976075
Akaike Information Criterion    149961.952149
Bayesian Information Criterion  150008.468477
asc_cycle                           -3.798133
asc_drive                           -1.223374
asc_pt                              -0.503663
beta_cost                           -0.171365
beta_time                           -5.310159
{'Model0': <biogeme.results.bioResults object at 0x7f9d2bb69850>}


# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [112]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [113]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [114]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
results_1=estimate_results(biogeme_model1,'Model1')
print(all_results)

[15:01:59] < Warning >   You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]
                                       Model0         Model1
Number of estimated parameters       5.000000       5.000000
Sample size                      81086.000000   81086.000000
Final log likelihood            -74975.976075  -74975.976075
Akaike Information Criterion    149961.952149  149961.952149
Bayesian Information Criterion  150008.468477  150008.468477
asc_cycle                           -3.798133      -3.798133
asc_drive                           -1.223374      -1.223374
asc_pt                              -0.503663      -0.503663
beta_cost                           -0.171365      -0.171365
beta_time                           -5.310159      -5.310159
{'Model0': <biogeme.results.bioResults object at 0x7f9d2bb69850>, 'Model1': <biogeme.results.bioResults object at 0x7f9d46a98f70>}


### Comparaison

we use log-likehood ratio as the model 0 is a reduction of the model 1

In [115]:
ratio_test(results_0,results_1,0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=10247.14611624839, threshold=7.814727903251179)

In [116]:
download_results()